In [ ]:
# Imports

import numpy as np
import random
import cv2
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SimpleRNN, GRU, LSTM, Dense, Flatten, TimeDistributed

In [ ]:
# Parameters setting

dataset_size = 10
num_frames = 45
frame_width = 90
frame_height = 45
step = 2

batch_size = 32
epochs = 5

labels = []
data = []
dataset = []

### Generate Data

In [ ]:
def make_frame(label, init_x, init_y):
  frames = []
  next_y = init_y
  next_x = init_x

  for i in range(num_frames):

    frame = np.zeros((frame_height, frame_width))
    cv2.circle(frame, (next_y, next_x), 5, 255, -1) 

    next_x = random.randint(next_x - 2, next_x + 2)

    if label==1:
      next_y += step 

    else:
      next_y -= step

    frames.append(frame)

  return frames

In [ ]:
def generate_data(initial_state_x, random_num):

  if random_num < 0.5:
    label = 0
    initial_state_y = frame_width - 1

  else:
    label = 1
    initial_state_y = 1

  frames = make_frame(label, initial_state_x, initial_state_y)

  return frames, label

In [ ]:
def main(dataset_size):

  labels = []
  data = [] 
  dataset = []
  
  for d in range(dataset_size):
    random_num = random.random()
    initial_state_x = random.randint(2, frame_height - 2)

    data, label = generate_data(initial_state_x, random_num)

    dataset.append(data)
    labels.append(label)

  # Convert to np.array
  dataset = np.array(dataset)
  labels = np.array(labels)

  # Reshape dataset
  dataset = dataset[..., np.newaxis]
  labels = labels[..., np.newaxis]

  return dataset, labels

In [ ]:
# Call data generation function

dataset, labels = main(dataset_size)

In [ ]:
# Plot

for frame in range(len(dataset[8, :])):
    print("frame:", frame+1)
    cv2_imshow(dataset[0, frame, :, :])
    print("\n")

In [ ]:
print(labels)

[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [0]]


### Define Models, Compile and fit

01- RNN

In [ ]:
# RNN Model

rnn_model = tf.keras.models.Sequential([
                                    # CNN
                                    TimeDistributed(Conv2D(2, (3, 3), activation="relu", input_shape=(None, frame_height, frame_width, 1))),
                                    TimeDistributed(MaxPooling2D(pool_size=(2, 2))),

                                    TimeDistributed(Flatten()),

                                    # RNN
                                    SimpleRNN(50),
                                    Dense(2, activation="softmax")
])

In [ ]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

rnn_model.fit(dataset, labels, batch_size=batch_size, epochs=epochs)

Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 1.2231 - accuracy: 0.3000
Epoch 2/5
1/1 [==============================] - 0s 98ms/step - loss: 0.8839 - accuracy: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 173ms/step - loss: 0.7024 - accuracy: 0.6000
Epoch 4/5
1/1 [==============================] - 0s 210ms/step - loss: 0.5292 - accuracy: 0.7000
Epoch 5/5
1/1 [==============================] - 0s 131ms/step - loss: 0.4020 - accuracy: 0.7000


In [ ]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 45, 43, 88, 2)    20        
 ibuted)                                                         
                                                                 
 time_distributed_1 (TimeDis  (None, 45, 21, 44, 2)    0         
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 45, 1848)         0         
 tributed)                                                       
                                                                 
 simple_rnn (SimpleRNN)      (None, 50)                94950     
                                                                 
 dense (Dense)               (None, 2)                 102       
                                                        

#### 02- GRU

In [ ]:
# GRU Model

gru_model = tf.keras.models.Sequential([
                                    # CNN
                                    TimeDistributed(Conv2D(2, (3, 3), activation="relu", input_shape=(None, frame_height, frame_width, 1))),
                                    TimeDistributed(MaxPooling2D(pool_size=(2, 2))),

                                    TimeDistributed(Flatten()),

                                    # GRU
                                    GRU(50),
                                    Dense(2, activation="softmax")
])

In [ ]:
gru_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

gru_model.fit(dataset, labels, batch_size=batch_size, epochs=epochs)

Epoch 1/5
1/1 [==============================] - 4s 4s/step - loss: 1.3150 - accuracy: 0.3000
Epoch 2/5
1/1 [==============================] - 0s 37ms/step - loss: 0.3369 - accuracy: 0.9000
Epoch 3/5
1/1 [==============================] - 0s 33ms/step - loss: 0.1748 - accuracy: 0.9000
Epoch 4/5
1/1 [==============================] - 0s 33ms/step - loss: 0.1059 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 33ms/step - loss: 0.0907 - accuracy: 1.0000


In [ ]:
gru_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_3 (TimeDis  (None, 45, 43, 88, 2)    20        
 tributed)                                                       
                                                                 
 time_distributed_4 (TimeDis  (None, 45, 21, 44, 2)    0         
 tributed)                                                       
                                                                 
 time_distributed_5 (TimeDis  (None, 45, 1848)         0         
 tributed)                                                       
                                                                 
 gru (GRU)                   (None, 50)                285000    
                                                                 
 dense_1 (Dense)             (None, 2)                 102       
                                                      

#### 03- LSTM

In [ ]:
# LSTM Model

lstm_model = tf.keras.models.Sequential([
                                    # CNN
                                    TimeDistributed(Conv2D(2, (3, 3), activation="relu", input_shape=(None, frame_height, frame_width, 1))),
                                    TimeDistributed(MaxPooling2D(pool_size=(2, 2))),

                                    TimeDistributed(Flatten()),

                                    # LSTM
                                    LSTM(50),
                                    Dense(2, activation="softmax")
])

In [ ]:
lstm_model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

lstm_model.fit(dataset, labels, batch_size=batch_size, epochs=epochs)

Epoch 1/5
1/1 [==============================] - 3s 3s/step - loss: 0.7277 - accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 34ms/step - loss: 0.4499 - accuracy: 0.8000
Epoch 3/5
1/1 [==============================] - 0s 36ms/step - loss: 0.3193 - accuracy: 0.8000
Epoch 4/5
1/1 [==============================] - 0s 32ms/step - loss: 0.2364 - accuracy: 1.0000
Epoch 5/5
1/1 [==============================] - 0s 30ms/step - loss: 0.1765 - accuracy: 1.0000


In [ ]:
lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_6 (TimeDis  (None, 45, 43, 88, 2)    20        
 tributed)                                                       
                                                                 
 time_distributed_7 (TimeDis  (None, 45, 21, 44, 2)    0         
 tributed)                                                       
                                                                 
 time_distributed_8 (TimeDis  (None, 45, 1848)         0         
 tributed)                                                       
                                                                 
 lstm (LSTM)                 (None, 50)                379800    
                                                                 
 dense_2 (Dense)             (None, 2)                 102       
                                                      

### Inference

In [ ]:
# Generate data for inference

labels = []
data = []
dataset = []

dataset, labels = main(dataset_size)

In [ ]:
rnn_acc = rnn_model.evaluate(dataset, labels)
gru_acc = gru_model.evaluate(dataset, labels)
lstm_acc = lstm_model.evaluate(dataset, labels)

print("accuracy of rnn model: ", rnn_acc[1])
print("accuracy of gru model: ", gru_acc[1])
print("accuracy of lstm model: ", lstm_acc[1])

1/1 [==============================] - 1s 572ms/step - loss: 0.6141 - accuracy: 0.7000
accuracy of rnn model:  0.5
accuracy of gru model:  0.800000011920929
accuracy of lstm model:  0.699999988079071
